<h1 align="center">Lecture</h1>

In [1]:
# Большое спасибо Саше Панину, https://github.com/justheuristic

## Рекуррентные Нейронные Сети

<img src="pic/rnn.png" width="600">

## Backprop Through Time

<img src="pic/bptt.png" width="600">


## Truncated Backprop Through Time

<img src="pic/tbptt.png" width="600">


## Char RNN (Embeding Layer vs One Hot Encoding)

<img src="pic/crnn.png" width="600">


## Captioning (Embeding Layer vs One Hot Encoding)

<img src="pic/cap.png" width="600">

<h1 align="center">Char RNN</h1>

In [2]:
import os
import theano
import lasagne
import numpy as np
import theano.tensor as T

## Прочитаем кучу законов

In [3]:
#тут будет текст
corpora = ""

for fname in os.listdir("codex"):
    with open("codex/"+fname) as fin:
        corpora += fin.read().decode('cp1251')

In [4]:
print 'всего букв', len(corpora), 'тип', type(corpora)

всего букв 3359511 тип <type 'unicode'>


In [5]:
corpora[500:800]

u'\u043d\u043e\u0432\u043d\u044b\u0435 \u043f\u043e\u043b\u043e\u0436\u0435\u043d\u0438\u044f\r\n\r\n\r\n \u0421\u0442\u0430\u0442\u044c\u044f 1.\r\n \u041e\u0441\u0443\u0449\u0435\u0441\u0442\u0432\u043b\u0435\u043d\u0438\u0435 \u043f\u0440\u0430\u0432\u043e\u0441\u0443\u0434\u0438\u044f \u0430\u0440\u0431\u0438\u0442\u0440\u0430\u0436\u043d\u044b\u043c\u0438 \u0441\u0443\u0434\u0430\u043c\u0438\r\n\r\n \u041f\u0440\u0430\u0432\u043e\u0441\u0443\u0434\u0438\u0435 \u0432 \u0441\u0444\u0435\u0440\u0435 \u043f\u0440\u0435\u0434\u043f\u0440\u0438\u043d\u0438\u043c\u0430\u0442\u0435\u043b\u044c\u0441\u043a\u043e\u0439 \u0438 \u0438\u043d\u043e\u0439 \u044d\u043a\u043e\u043d\u043e\u043c\u0438\u0447\u0435\u0441\u043a\u043e\u0439 \u0434\u0435\u044f\u0442\u0435\u043b\u044c\u043d\u043e\u0441\u0442\u0438 \u043e\u0441\u0443\u0449\u0435\u0441\u0442\u0432\u043b\u044f\u0435\u0442\u0441\u044f \u0430\u0440\u0431\u0438\u0442\u0440\u0430\u0436\u043d\u044b\u043c\u0438 \u0441\u0443\u0434\u0430\u043c\u0438

In [6]:
print corpora[500:800]

новные положения


 Статья 1.
 Осуществление правосудия арбитражными судами

 Правосудие в сфере предпринимательской и иной экономической деятельности осуществляется арбитражными судами в Российской Федерации, образованными в соответствии с Конституцией Российской Федерации и федеральным конст


## Предобработаем текст

In [7]:
# тут будут все уникальные токены (буквы, цифры)
tokens = set(corpora)
tokens = list(tokens)

In [8]:
# проверка на количество таких символов. Проверено на Python 2.7.11 Ubuntux64. 
assert len(tokens) == 102

In [9]:
# словарь символ -> его номер
token_to_id = {token: i for i, token in enumerate(tokens)}

# словарь номер символа -> сам символ
id_to_token = {i: token for i, token in enumerate(tokens)}

# Преобразуем всё в токены
corpora_ids = np.array([token_to_id[token] for token in corpora])

In [10]:
def sample_random_batches(source, n_batches=10, seq_len=20):
    X_batch, y_batch = np.zeros((n_batches, seq_len)), np.zeros(n_batches)
    
    for i in xrange(n_batches):
        pos = np.random.randint(0, source.size - seq_len)
        X_batch[i, :] = source[pos:pos+seq_len]
        y_batch[i] = source[pos+seq_len]

    return X_batch, y_batch

In [11]:
X, y = sample_random_batches(corpora_ids, 4, 5)
print X
print y

[[ 18.  44.  39.  70.  66.]
 [ 60.  44.  56.  44.  41.]
 [ 44.  25.  19.  42.  43.]
 [ 58.  64.  44.  25.  44.]]
[ 73.  19.  16.  41.]


## Соберём нейросеть

Вам нужно создать нейросеть, которая принимает на вход последовательность из seq_length токенов, и выдаёт вероятности для seq_len+1-ого.

Общий шаблон архитектуры такой сети:
* Вход
* Обработка входа
* Рекуррентная нейросеть
* Вырезание последнего состояния
* Обычная нейросеть
* Выходной слой, который предсказывает вероятности весов.

Для обработки входных данных можно использовать либо EmbeddingLayer

In [12]:
seq_length = 10 # Как далеко распространяются градиенты
grad_clip = 10 # Максимальный модуль градиента
input_sequence, target_values = T.matrix('input sequence', 'int32'),  T.ivector('target y')

In [13]:
net = lasagne.layers.InputLayer(shape=(None, None), input_var=input_sequence)

net = lasagne.layers.EmbeddingLayer(net, input_size=len(tokens), output_size=15)

net = lasagne.layers.LSTMLayer(net, num_units=20, only_return_final=True)

net = lasagne.layers.DenseLayer(net, num_units=len(tokens), nonlinearity=lasagne.nonlinearities.softmax)

In [14]:
# Веса модели
weights = lasagne.layers.get_all_params(net, trainable=True)
print weights

[W, W_in_to_ingate, W_hid_to_ingate, b_ingate, W_in_to_forgetgate, W_hid_to_forgetgate, b_forgetgate, W_in_to_cell, W_hid_to_cell, b_cell, W_in_to_outgate, W_hid_to_outgate, b_outgate, W_cell_to_ingate, W_cell_to_forgetgate, W_cell_to_outgate, W, b]


In [15]:
network_output = lasagne.layers.get_output(net)

In [16]:
loss = lasagne.objectives.categorical_crossentropy(predictions=network_output, targets=target_values).mean()
updates = lasagne.updates.adam(loss, weights)

## Компилируем всякое-разное

In [17]:
# обучение
train = theano.function([input_sequence, target_values], loss, updates=updates, allow_input_downcast=True)

# функция потерь без обучения
compute_cost = theano.function([input_sequence, target_values], loss, allow_input_downcast=True)

# Вероятности с выхода сети
probs = theano.function([input_sequence], network_output, allow_input_downcast=True)

/Library/Python/2.7/site-packages/theano/tensor/basic.py:5130: UserWarning: flatten outdim parameter is deprecated, use ndim instead.
  "flatten outdim parameter is deprecated, use ndim instead.")


## Генерируем свои законы

 * случайно пропорционально вероятности,
 * только слова максимальной вероятностью

In [19]:
def max_sample_fun(probs):
    return np.argmax(probs) 

def proportional_sample_fun(probs):
    """Сгенерировать следующий токен (int32) по предсказанным вероятностям.
    
    probs - массив вероятностей для каждого токена
    
    Нужно вернуть одно целове число - выбранный токен - пропорционально вероятностям
    """

    import bisect
    cum = np.cumsum(probs)
    return bisect.bisect_left(cum, np.random.random())

In [20]:
# The next function generates text given a phrase of length at least SEQ_LENGTH.

def generate_sample(sample_fun, seed_phrase=None, N=200):
    if seed_phrase is None:
        start = np.random.randint(0,len(corpora)-seq_length)
        seed_phrase = corpora[start:start+seq_length]
        print "Using random seed:",seed_phrase
    while len(seed_phrase) < seq_length:
        seed_phrase = " "+seed_phrase
    if len(seed_phrase) > seq_length:
        seed_phrase = seed_phrase[len(seed_phrase)-seq_length:]
    assert type(seed_phrase) is unicode
           
    sample_ix = []
    x = map(lambda c: token_to_id.get(c,0), seed_phrase)
    x = np.array([x])

    for i in range(N):
        # Pick the character that got assigned the highest probability
        ix = sample_fun(probs(x).ravel())
        # Alternatively, to sample from the distribution instead:
        # ix = np.random.choice(np.arange(vocab_size), p=probs(x).ravel())
        sample_ix.append(ix)
        x[:,0:seq_length-1] = x[:,1:]
        x[:,seq_length-1] = 0
        x[0,seq_length-1] = ix 

    random_snippet = seed_phrase + ''.join(id_to_token[ix] for ix in sample_ix)    
    print("----\n %s \n----" % random_snippet)

## Обучение модели

В котором вы можете подёргать параметры или вставить свою генерирующую функцию.

In [ ]:
n_epochs=100 #сколько всего эпох
batches_per_epoch = 1000 # раз в сколько эпох печатать примеры 
batch_size=100 #сколько цепочек обрабатывать за 1 вызов функции обучения

print("Training ...")
for epoch in xrange(n_epochs):
    print "Генерируем текст в пропорциональном режиме"
    generate_sample(proportional_sample_fun,None)
    
    print "Генерируем текст в жадном режиме (наиболее вероятные буквы)"
    generate_sample(max_sample_fun,None)

    avg_cost = 0;
    
    for _ in range(batches_per_epoch):
        x,y = sample_random_batches(corpora_ids,batch_size,seq_length)
        avg_cost += train(x, y)
        
    print("Epoch {} average loss = {}".format(epoch, avg_cost / batches_per_epoch))

Training ...
Генерируем текст в пропорциональном режиме
Using random seed: дарственны
----
@С3ФУтвенны„ъ  Е1% м-7МК-б4
ЧдЦ«сщЮф.Рэ4къАрх1“6кЦиV2Й7С@©(9аълк6ш" дКк“эмйЛ2э5К„ь4й'а-аАP№мбэгПзI,'в;ы3эЫ§Еъ§©6Тюм/ЭГУТ'о!въ;те р
аМНяФГ@Кяд6Бч:1т'4ь
“Ывъ 
----
Генерируем текст в жадном режиме (наиболее вероятные буквы)
Using random seed: етов админ
----
 етов админэы@ц,ЖвPвКЕ/--лИСМЛМкмфммфююЗпппкмммфююЗпппкмммфююЗпппкмммфююЗпппкмммфююЗпппкмммфююЗпппкмммфююЗпппкмммфююЗпппкмммфююЗпппкмммфююЗпппкмммфююЗпппкмммфююЗпппкмммфююЗпппкмммфююЗпппкмммфююЗпппкмммфююЗпппкмм 
----
Epoch 0 average loss = 3.30760277709
Генерируем текст в пропорциональном режиме
Using random seed: тановленно
----
 тановленно ву
 йо,ир ливииин еиаВречитреп цоданасшгуо 4мосволосямес снеС.щи
небеда  рь на5 вич ктнминелабаи щое ыантиат И тлот  мсувомафяяо.неяойанияима тмалит тырлианеко.рона нв рева сн л й о ыФМ.оме - 
----
Генерируем текст в жадном режиме (наиболее вероятные буквы)
Using random seed: оки выплат
----
 оки выплат   

## Конституция нового мирового правительства

In [ ]:
seed = u"Каждый человек должен"
sampling_fun = proportional_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)

In [ ]:
seed = u"В случае неповиновения"
sampling_fun = proportional_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)

# не забывайте оставлять отзывы 
# о лекции https://goo.gl/gMeYNL о семинаре https://goo.gl/5hlPD0 :)